In [53]:
import nltk, re, pprint
import pandas as pd
import numpy as np

### 1. Import & QA

In [54]:
data = pd.read_csv("D:\ML\Datasets\labeled_lyrics_cleaned.csv")

In [55]:
data.head()

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.63
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.63
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.24
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.54
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.37


In [56]:
data.drop(labels="Unnamed: 0", axis=1, inplace=True)
data.rename(columns={"seq": "lyrics", "label": "valency"}, inplace=True)

#data = data.drop_duplicates(subset=['lyrics', 'song'])
data.sort_values(by=['song', 'valency'], ascending=False, inplace=True) # to keep highest value valency 
data = data.drop_duplicates(subset='lyrics')
pd.set_option('display.float_format', lambda x: '%.2f' % x) # round everything to 2 decimal places
data = data.reset_index() 
data.head()

,index,artist,lyrics,song,valency
0,54911,Simon & Milo,"Hello, this is Stacy, the computer\nGood morni...",www.nevergetoveryou,0.68
1,82479,Hippo Campus,See how the western kids\r\nHave silicon insid...,western kids,0.52
2,82478,Hippo Campus,"Wisconsin pines, collaborating with the day gl...",way it goes,0.52
3,82477,Hippo Campus,"I see meaning where you don't, where you don't...",vines,0.66
4,82476,Hippo Campus,My thoughts are a battlefield of sub-surreal a...,vacation,0.55


### 2 EDA

In [70]:
lyrics_all_raw = [lyrics for lyrics in data['lyrics']]
lyrics_all_raw = ''.join(lyrics_all_raw[:1000])
text1 = data['lyrics'][3335]
fdist1 = nltk.FreqDist(text1)

In [71]:
def lexical_diversity(text):
    return len(text) / len(set(text))

def percentage(count, total):
    return 100 * count / total

lexical_diversity(text1), percentage(text1.count('my'), len(text1))

(27.071428571428573, 0.19788918205804748)

In [59]:
from nltk import tokenize
from nltk.corpus import brown, stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import webtext

lyrics = [x.replace("\n", " ").strip().replace("\r", "") for x in data['lyrics']]

In [60]:
#lyrics

In [176]:
import string # removing punctuation

pattern = r'\[.*?\]' # remove square brackets and contents

song = data['lyrics'][3336].replace("\n", " ").replace("\r", "").replace("\t", "").replace("  ", " ").strip()
song = song.translate(str.maketrans('', '', string.punctuation))
song = re.sub(pattern, '', song)
song

'The words of wordless may Sing a song to me She stands as tall as trees these days The words of wordless may  Dear Jesus make me simple Strong as trees to sway Give me arms wide open with your beautiful way Just like wordless may  If there were fields as pretty as you Id want to kiss them too If there were fields as pretty as you Id want to kiss them too  If there were eyes as pretty and blue Id want to swim them too If there were skys as pretty and blue Id want to kiss them too  The words of wordless may Sing a song to me She stands as tall as trees these days The words of wordless may Dear Jesus make me simple Strong as trees to sway Give me arms wide open with your beautiful way Just like wordless may  If there were fields as pretty as you Id want to kiss them too If there were skys as pretty as you Id want to kiss them too Music The words of wordless may Sing a song to me She stands as tall as trees these days The words of wordless may Dear Jesus make me simple Strong as trees to 

In [177]:
from nnsplit import NNSplit

splitter = NNSplit.load("en")
                        #, stride=7, padding=20, threshold=0.8)

In [178]:
help(NNSplit.load)

Help on built-in function load:

load(model_name, use_cuda=None, **kwargs)
    Loads a built-in model. From the local cache or from the interent if it is not cached.
    
    Args:
        model_name (str): Name of the model.
        use_cuda (bool): Whether to use CUDA to run the model on GPU. If None, will use CUDA if available, otherwise CPU.
        **kwargs: Additional options. Can be:
            * threshold (float): Threshold from 0 to 1 above which predictions will be considered positive.
            * stride (int): How much to move the window after each prediction (comparable to stride of 1d convolution).
            * max_length (int): The maximum length of each cut (comparable to kernel size of 1d convolution).
            * padding (int): How much to zero pad the text on both sides.
            * batch_size (int): Batch size to use.
            * length_divisor (int): Total length will be padded until it is divisible by this number. Allows some additional optimizations.



In [181]:
splits = splitter.split([song.lower()])[0]
song_sents = [str(x).strip() for x in splits]
#song_sents = sorted(set(song_sents))

In [182]:
song_sents

['the words of wordless may sing a song to me',
 'she stands as tall as trees',
 'these days the words of wordless may',
 'dear jesus make me simple strong as trees to sway give me arms wide open with your beautiful way just like wordless may',
 'if there were fields as pretty as you id want to kiss them too if there were fields as pretty as you id want to kiss them too',
 'if there were eyes as pretty and blue id want to swim them too if there were skys as pretty and blue id want to kiss them too',
 'the words of wordless may sing a song to me',
 'she stands as tall as trees these days',
 'the words of wordless may dear',
 'jesus make me simple strong as trees to sway give me arms wide open with your beautiful way just like wordless may',
 'if there were fields as pretty as you id want to kiss them too if there were skys as pretty as you id want to kiss them too music',
 'the words of wordless may sing a song to me',
 'she stands as tall as trees these days',
 'the words of wordless m

In [64]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()    
tokens = [tokenize.word_tokenize(sent) for sent in song_sents] # should be no punct

In [65]:
from nltk import pos_tag, word_tokenize
pos_tag(word_tokenize(song_sents[0])) 

[('headz', 'NN')]

In [66]:
song = song.translate(str.maketrans('', '', string.punctuation))
#pos_tag(word_tokenize(song))

In [67]:
from textblob import TextBlob
from textblob.parsers import PatternParser

b = TextBlob(song_sents[0])
print(b.parse())

headz/NN/B-NP/O


In [68]:
import spacy
#import textacy

nlp = spacy.load('en_core_web_sm')

In [69]:
doc = nlp(song_sents[0])

for x in doc:
    print(x, spacy.explain(x.tag_))

headz adjective (English), other noun-modifier (Chinese)
